In [ ]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s\t%(name)s\t%(levelname)s\t%(message)s")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import obspy
from obspy import UTCDateTime as utc
from obspy.clients.fdsn import Client
from obspy.core.event import Catalog
from eqcorrscan.utils.catalog_utils import filter_picks
from eqcorrscan.core import template_gen as template_gen
client = Client('IRIS')
from eqcorrscan import Tribe

run_name='adams'
upath='/Users/bnjo/home/bnjo/'+run_name+'/util/' # util directory path
tpath='/Users/bnjo/home/bnjo/'+run_name+'/detect/tribes/' #Outpath for tribe when sving.
qpath = '/Users/bnjo/home/bnjo/'+run_name+'/util/'+run_name+'/' #path to the QuakeML Files.


In [ ]:
# Various functions to generate the catalog and verify the events that will be used as templates 

def get_catalog(run_name):
    catalog = Catalog()
    for e in evids:
        e = str(e)+'.xml'        
        event = obspy.read_events(qpath+e)
        catalog += event

    return catalog

#Make sure there are phase hints for the phases (P or S) you want to use becausae eqcorrscan needs this. 
def clean_catalog(catalog):
    for event in catalog.events:
        try:
            event = event.get_preferred_origin()
        except:
            event = event.origins[0]
        arrivals = event.arrivals
        for i in range(len(arrivals)):
            arrival = arrivals[i]
            ph = arrival.phase
            pick = arrival.pick_id.get_referred_object()
            if pick != None:
                pick.phase_hint = str(ph)
    return catalog

#Function to generate the tribes based on the events that are in the catalog. 
def tribe_gen(catalog):
    tribe = Tribe().construct(
        method="from_client",
        lowcut=1.0,
        highcut=10.0,
        samp_rate=50.0,
        length=5,
        filt_order=4,
        prepick=1,
        delayed=True,
        swin='P_all',
        client_id=client,
        catalog=catalog,
        data_pad=10.,
        process_len=3600,
        parallel=True)
    return tribe


# Function to rename the templates within the tribe using their UW Event ID. 
def rename_templates(tribe):
    for template in tribe:
        event = template.event
        name = str(event.resource_id).split(sep='/')[-1]
        print(name)
        template.name = name
    return tribe


In [ ]:
## Generate the tribe by reading the QuakeML Files downloaded to disk locally from sarchdb.
## First, generate the catalog, then clean up the catalog (make sure phase hints are in there)
## Next (optional), filter the stations and channels of the templates to use certain ones. 
# Generate the tribe
# Rename the templates in the tribe(optional)
# Save the tribe (optional). easy to load from saved file; doesn;t require downloading the trace data again. , I would recommend


c = get_catalog(run_name)
c = clean_catalog(c)
stations = ['ASCR','ASR2','ATOM']
c = filter_picks(c,stations=stations)
tribe = tribe_gen(c)
tribe = rename_templates(tribe)
tribe.write(tpath+run_name,compress=True,catalog_format='QUAKEML')


In [ ]:
# You can run the detector here too to check and see how it went. 


from obspy import UTCDateTime as utc
party= tribe.client_detect(
    client=client,
    starttime = utc('2024-10-05T01:00:00.000000Z'),
    endtime = utc('2024-10-11T00:00:00.000000Z'),
    threshold = 10,
    threshold_type = "MAD",
    trig_int = 6.0,
    ignore_bad_data = True,
    return_stream=False,
    concurrency='multiprocess',
    parallel = True,
    num_cores = 10,
    concurrent_processing=True,
    group_size = 3)



In [ ]:
party